In [7]:
from torchvision import datasets
from torchvision.transforms import Compose, ToTensor

# On définit d'abord un transform minimal (on ajoutera resize + padding ensuite)
base_transform = Compose([
    ToTensor(),  # convertit en [0,1] C×H×W
])

dataset = datasets.ImageFolder(
    root="/home/mateo/Travail/Fine-tune/coca-ofb/data",
    transform=base_transform
)

from torch.utils.data import DataLoader
loader = DataLoader(
    dataset,
    batch_size=16,       # à adapter
    shuffle=True,
    num_workers=4
)


FileNotFoundError: Couldn't find any class folder in /home/mateo/Travail/Fine-tune/coca-ofb/data.